In [1]:
import re
import time
import pickle
import requests
import subprocess

In [2]:
# Question 1

!uv --version

uv 0.9.5


In [3]:
# Question 2

with open("uv.lock") as f:
    content = f.read()

package_block = re.search(
    r'\[\[package\]\]\s+name\s*=\s*"scikit-learn"([\s\S]*?)(?=\[\[package\]\]|$)',
    content
)

if package_block:
    hash_match = re.search(r'sha256:[a-f0-9]+', package_block.group(1))
    if hash_match:
        print(hash_match.group(0))
    else:
        print("No hash found in scikit-learn block")
else:
    print("scikit-learn not found in uv.lock")

sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e


In [4]:
# Model Load

url = "https://github.com/DataTalksClub/machine-learning-zoomcamp/raw/refs/heads/master/cohorts/2025/05-deployment/pipeline_v1.bin"

response = requests.get(url)
response.raise_for_status()  # ensure it downloaded OK

dv, model = pickle.loads(response.content)

c:\Users\Luis\miniconda3\envs\mlcamp\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Luis\miniconda3\envs\mlcamp\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\Luis\miniconda3\envs\mlcamp\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from version 1.6.1 when using version 

In [5]:
# Question 3

record = {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

X = dv.transform(record)

model.predict_proba(X)[0, 1]

np.float64(0.5336072702798061)

In [6]:
# Question 4

process = subprocess.Popen(
    ["python", "-m", "uvicorn", "app:app", "--host", "0.0.0.0", "--port", "9696"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
)
time.sleep(3)

url = "http://127.0.0.1:9696/predict"

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
predictions = response.json()

print(predictions)

{'churn_probability': 0.5340417283801275, 'churn': True}


In [7]:
# Question 5

!docker pull agrigorev/zoomcamp-model:2025
!docker images agrigorev/zoomcamp-model:2025

2025: Pulling from agrigorev/zoomcamp-model
Digest: sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
Status: Image is up to date for agrigorev/zoomcamp-model:2025
docker.io/agrigorev/zoomcamp-model:2025
REPOSITORY                 TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-model   2025      14d79fde0bbf   6 days ago   181MB


In [8]:
# Building the Docker image and starting the container

!docker build -t customer-scoring-service:latest .
!docker run -d -p 9696:9696 customer-scoring-service:latest

#0 building with "desktop-linux" instance using docker driver

#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 397B done
#1 DONE 0.0s

#2 [internal] load metadata for docker.io/agrigorev/zoomcamp-model:2025
#2 DONE 0.0s

#3 [internal] load .dockerignore
#3 transferring context: 2B done
#3 DONE 0.0s

#4 [internal] load build context
#4 transferring context: 63B done
#4 DONE 0.0s

#5 [1/5] FROM docker.io/agrigorev/zoomcamp-model:2025@sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae
#5 resolve docker.io/agrigorev/zoomcamp-model:2025@sha256:14d79fde0bbf078eb18c99c2bd007205917b758ec11060b2994963a1e485c2ae 0.0s done
#5 DONE 0.0s

#6 [2/5] WORKDIR /app
#6 CACHED

#7 [3/5] RUN pip install --upgrade pip     && pip install fastapi==0.120.1 scikit-learn==1.6.1 uvicorn==0.38.0
#7 CACHED

#8 [4/5] COPY pipeline_v1.bin /app/pipeline_v1.bin
#8 CACHED

#9 [5/5] COPY app.py /app/app.py
#9 CACHED

#10 exporting to image
#10 exporting layers 0.0s don

5ff9351e6b8958ce7bc4fa80c700823119cceb3d7ddb81c996fd37c1e9d569e4


In [9]:
# Question 6


url = url = "http://127.0.0.1:9696/predict"

client = {
    "lead_source": "organic_search",
    "number_of_courses_viewed": 4,
    "annual_income": 80304.0
}

response = requests.post(url, json=client)
print(response.json())

{'churn_probability': 0.5340417283801275, 'churn': True}


In [10]:
!for /f %i in ('docker ps -q') do docker stop %i


(mlcamp) c:\Users\Luis\Desktop\workspace\ml-zcamp\05-deployment>docker stop 5ff9351e6b89 
5ff9351e6b89
